Signal 6 : Timeo Martin, Arno Braz, Julie Le Ral

Partie 2 - Reconnaissance automatique de voyelle 

In [22]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal

signal_dict = np.load('../Sons/apprentissage/a_1.npz') 
# signal_dict est un dictionnaire → contenant les données contenues dans le fichier signal.npz

1. Lecture des fichiers et affichages

In [23]:
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

# Signal chargé en fonction du temps
trace_signal = px.line(x=t, y=s, labels={'x': 'Temps (t)', 'y': 's'}).show()

# Calculer la Transformée de Fourier du signal
tf = san.trans_fourier(s, t)
trace_spectre = px.line(x=tf[0], y=abs(tf[1]), labels={'x': 'frequences (nu)', 'y': 'S'}).show()

# Logarithme du spectre
trace_log10_spectre = px.line(x=tf[0], y=np.log10(abs(tf[1])), labels={'x': 'frequences (nu)', 'y': 'S'}).show()

T = (0.5-0.36)/15 = 0.00933s 

nu0 = 107.2Hz -> Correspond à l'oObservation graphique  

C'est plus lisible en échelle logarithmique et donc plus adapté (notament pour les hautes fréquences) pour lire les harmoniques distantes de la fondamentale.


2. Apprentissage - Caractérisation des voyelles

In [24]:
def harmonique(signal, H) : 
    signal_dict = np.load(signal) 
    s = signal_dict['s'] # voyelle utile 
    t = signal_dict['t']

    # Calculer la Transformée de Fourier du signal
    nu,g = san.trans_fourier(s, t)
    spectre = abs(g) ** 2
    spectre_normal = spectre / (max(spectre))

    # Fondamental
    fondamental = san.detect_fondamentale(spectre_normal, nu) 

    # Création du tableau des fréquences approximative des harmoniques
    frequence_h = []
    for i in range(H) :
        frequence_h.append((1 + i) * fondamental)

    harmonique_freq, harmonique_amp = san.detect_pics(spectre, frequence_h, nu)

    res = []
    for i in range(H) : 
        amplitude = harmonique_amp[i]
        amplitude = abs(amplitude) + 1e-12 # éviter log(0)
        res.append((float(np.log10(amplitude))))

    print("Vecteur harmonique (log10 amplitudes) :", res)

    # fig_temporel = px.line(x = t, y = s, title = "Signal Temporel").show()
    # fig_fourier = px.line(x = nu, y = spectre, title = "Spectre de Fourier").show()

harmonique('../Sons/apprentissage/a_1.npz', 10)
harmonique('../Sons/apprentissage/a_2.npz', 10)
harmonique('../Sons/apprentissage/a_3.npz', 10)
harmonique('../Sons/apprentissage/a_4.npz', 10)
harmonique('../Sons/apprentissage/a_5.npz', 10)


Vecteur harmonique (log10 amplitudes) : [5.287575721740723, 5.0999064445495605, 3.9779834747314453, 4.049320220947266, 3.6778976917266846, 3.7982685565948486, 4.44309663772583, 4.9137396812438965, 4.309008598327637, 3.115867853164673]
Vecteur harmonique (log10 amplitudes) : [5.495128631591797, 5.2783918380737305, 4.154568672180176, 4.705029010772705, 4.08192777633667, 5.019125461578369, 4.9222846031188965, 4.423004150390625, 3.7212541103363037, 3.9053850173950195]
Vecteur harmonique (log10 amplitudes) : [5.193875789642334, 4.928432464599609, 3.918626546859741, 4.395035743713379, 4.751338481903076, 4.994625091552734, 4.321593761444092, 3.7601828575134277, 4.071331024169922, 3.247727155685425]
Vecteur harmonique (log10 amplitudes) : [5.331789970397949, 5.17434549331665, 3.4973833560943604, 3.6671829223632812, 2.98946475982666, 4.457493782043457, 4.481418609619141, 4.604557037353516, 3.8776090145111084, 4.359222888946533]
Vecteur harmonique (log10 amplitudes) : [4.958671569824219, 4.67348

2.3 Test de la méthode